# 分层索引在文档检索中的应用

## 概览

这段代码实现了一个分层索引系统用于文档检索，采用两个级别的编码：文档级别的摘要和详细的片段。这种方法旨在通过首先通过摘要识别相关文档部分，然后深入到这些部分的具体细节中，来提高信息检索的效率和相关性。

## 动机

传统的平面索引方法可能在处理大型文档或语料库时遇到困难，可能会错过上下文或返回不相关信息。分层索引通过创建一个两级搜索系统来解决这个问题，允许更有效和上下文感知的检索。

## 关键组成部分

1. PDF处理和文本分块
2. 使用OpenAI的GPT-4异步文档摘要
3. 使用FAISS和OpenAI嵌入创建摘要和详细片段的向量存储
4. 自定义的分层检索函数

## 方法细节

### 文档预处理和编码

1. 加载PDF并将其分割成文档（可能按页面）。
2. 每个文档都使用GPT-4异步摘要。
3. 原始文档也被分割成更小的详细片段。
4. 创建两个单独的向量存储：
   - 一个用于文档级别的摘要
   - 一个用于详细片段

### 异步处理和速率限制

1. 代码使用异步编程（asyncio）来提高效率。
2. 实现批处理和指数退避来处理API速率限制。

### 分层检索

`retrieve_hierarchical`函数实现了两级搜索：

1. 它首先搜索摘要向量存储以识别相关文档部分。
2. 对于每个相关摘要，然后搜索详细片段向量存储，按相应的页码过滤。
3. 这种方法确保只从最相关的文档部分检索详细信息。

## 这种方法的好处

1. 提高检索效率：通过首先搜索摘要，系统可以快速识别相关文档部分，而无需处理所有详细片段。
2. 更好的上下文保留：分层方法有助于保持检索信息的更广泛上下文。
3. 可扩展性：这种方法特别适合大型文档或语料库，平面搜索可能效率低下或错过重要上下文。
4. 灵活性：系统允许调整检索的摘要和片段数量，可以为不同的用例进行微调。

## 实现细节

1. 异步编程：使用Python的asyncio进行高效的I/O操作和API调用。
2. 速率限制处理：实现批处理和指数退避，有效管理API速率限制。
3. 持久存储：将生成的向量存储本地保存，以避免不必要的重新计算。

## 结论

分层索引代表了一种复杂的文档检索方法，特别适合用于大型或复杂的文档集。通过利用高层次的摘要和详细的片段，它在广泛的上下文理解和特定信息检索之间提供了平衡。这种方法在需要高效和上下文感知的信息检索的各个领域都有潜在的应用，例如法律文件分析、学术研究或大规模内容管理系统。


<div style="text-align: center;">

<img src="../images/hierarchical_indices.svg" alt="hierarchical_indices" style="width:50%; height:auto;">
</div>

### Import libraries 

In [14]:
import asyncio
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains.summarize.chain import load_summarize_chain
from langchain.docstore.document import Document

# Load environment variables from a .env file
load_dotenv()

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from rag.helper_functions import *
from rag.evaluation.evalute_rag import *


# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document path

In [15]:
path = "../data/Understanding_Climate_Change.pdf"

### 函数编码到摘要和块级别，共享页面元数据

In [16]:
async def encode_pdf_hierarchical(path, chunk_size=1000, chunk_overlap=200, is_string=False):
    """
    异步地将PDF书籍编码为使用OpenAI嵌入的分层向量存储。
    包括具有指数退避的速率限制处理。
    
    参数：
        path: PDF文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。
    
    返回：
        包含两个FAISS向量存储的元组：
        1. 文档级摘要
        2. 详细块

    """
    
    # Load PDF documents
    if not is_string:
        loader = PyPDFLoader(path)
        documents = await asyncio.to_thread(loader.load)
    else:
        text_splitter = RecursiveCharacterTextSplitter(
            # Set a really small chunk size, just to show.
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
        documents = text_splitter.create_documents([path])


    # Create document-level summaries
    summary_llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
    summary_chain = load_summarize_chain(summary_llm, chain_type="map_reduce")
    
    async def summarize_doc(doc):
        """
        对单个文档进行摘要，并处理速率限制。
        
        参数：
            doc: 要进行摘要的文档。
        
        返回：
            一个摘要后的Document对象。
        """
        # 以指数退避的方式重试摘要
        summary_output = await retry_with_exponential_backoff(summary_chain.ainvoke([doc]))
        summary = summary_output['output_text']
        return Document(
            page_content=summary,
            metadata={"source": path, "page": doc.metadata["page"], "summary": True}
        )

    # Process documents in smaller batches to avoid rate limits
    batch_size = 5  # Adjust this based on your rate limits
    summaries = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        batch_summaries = await asyncio.gather(*[summarize_doc(doc) for doc in batch])
        summaries.extend(batch_summaries)
        await asyncio.sleep(1)  # Short pause between batches

    # Split documents into detailed chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    detailed_chunks = await asyncio.to_thread(text_splitter.split_documents, documents)

    # Update metadata for detailed chunks
    for i, chunk in enumerate(detailed_chunks):
        chunk.metadata.update({
            "chunk_id": i,
            "summary": False,
            "page": int(chunk.metadata.get("page", 0))
        })

    # Create embeddings
    embeddings = OpenAIEmbeddings()

    # Create vector stores asynchronously with rate limit handling
    async def create_vectorstore(docs):
        """
        Creates a vector store from a list of documents with rate limit handling.
        
        Args:
            docs: The list of documents to be embedded.
            
        Returns:
            A FAISS vector store containing the embedded documents.
        """
        return await retry_with_exponential_backoff(
            asyncio.to_thread(FAISS.from_documents, docs, embeddings)
        )

    # Generate vector stores for summaries and detailed chunks concurrently
    summary_vectorstore, detailed_vectorstore = await asyncio.gather(
        create_vectorstore(summaries),
        create_vectorstore(detailed_chunks)
    )

    return summary_vectorstore, detailed_vectorstore

### 如果向量存储不存在，则将PDF书籍编码为文档级摘要和详细块


In [17]:
if os.path.exists("../vector_stores/summary_store") and os.path.exists("../vector_stores/detailed_store"):
   embeddings = OpenAIEmbeddings()
   summary_store = FAISS.load_local("../vector_stores/summary_store", embeddings, allow_dangerous_deserialization=True)
   detailed_store = FAISS.load_local("../vector_stores/detailed_store", embeddings, allow_dangerous_deserialization=True)

else:
    summary_store, detailed_store = await encode_pdf_hierarchical(path)
    summary_store.save_local("../vector_stores/summary_store")
    detailed_store.save_local("../vector_stores/detailed_store")


### 根据摘要级别检索信息，然后从块级向量存储中检索信息，并根据摘要级别页面进行过滤。

In [18]:
def retrieve_hierarchical(query, summary_vectorstore, detailed_vectorstore, k_summaries=3, k_chunks=5):
    """

    执行使用查询的分层检索。
    
    参数：
        query: 搜索查询。
        summary_vectorstore: 包含文档摘要的向量存储。
        detailed_vectorstore: 包含详细块的向量存储。
        k_summaries: 要检索的顶级摘要的数量。
        k_chunks: 每个摘要要检索的详细块的数量。
    
    返回：
        一个相关详细块的列表。

    """
    
    # 检索前 k 个 summary
    top_summaries = summary_vectorstore.similarity_search(query, k=k_summaries)
    
    relevant_chunks = []
    for summary in top_summaries:
        # 对于每个 summary，检索相关的详细块
        page_number = summary.metadata["page"]
        page_filter = lambda metadata: metadata["page"] == page_number
        page_chunks = detailed_vectorstore.similarity_search(
            query, 
            k=k_chunks, 
            filter=page_filter
        )
        relevant_chunks.extend(page_chunks)
    
    return relevant_chunks

### 例子

In [19]:
query = "What is the greenhouse effect?"
results = retrieve_hierarchical(query, summary_store, detailed_store)

# Print results
for chunk in results:
    print(f"Page: {chunk.metadata['page']}")
    print(f"Content: {chunk.page_content}...")  # Print first 100 characters
    print("---")

Page: 0
Content: driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate.  
Fossil Fuels  
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today.  
Coal...
---
Page: 0
Content: Most of these climate changes are attributed to very small variations in 